# TL;DR
Dans ce projet, nous exploitons les données de la base ASRS couvrant la période de janvier 1988 à décembre 2020. Étant donné la limite de 5000 lignes par export imposée par la plateforme de téléchargement, les données ont été récupérées en plusieurs morceaux.

Dans ce notebook, nous concaténons ces morceaux, puis `nous ne conservons que les colonnes contenant moins de 5% de valeurs manquantes`. Sur les `126 colonnes initiales`, *`seules 14 colonnes respectent ce critère`*. Cette approche permet de réduire la complexité des données tout en conservant les informations essentielles pour notre analyse.

Le résultat de ce traitement est exporté au `format parquet` afin de faciliter le stockage et d'optimiser les performances lors des traitements ultérieurs.

In [ ]:
from glob import glob
import pandas as pd

In [100]:
data = [
    pd.read_csv(f, on_bad_lines="skip", header=1) for f in glob("../data/00_raw/*.csv")
]

In [40]:
df = pd.concat(data)

In [76]:
primary_df = df.dropna(thresh=201706, axis=1)

In [77]:
primary_df.head()

,ACN,Date,Locale Reference,State Reference,Aircraft Operator,Make Model Name,Flight Phase,Reporter Organization,Function,ASRS Report Number.Accession Number,Anomaly,Detector,Primary Problem,Narrative,Synopsis
0,1299097,201510,IND.Airport,IN,NaN,B767-300 and 300 ER,Descent,Government,Enroute,1299097,Deviation - Altitude Crossing Restriction Not ...,Person Air Traffic Control,Human Factors,The flight came over descending to FL240 flyin...,A ZID Controller and the two B767 pilots invol...
1,1299098,201510,ZZZ.TRACON,US,Military,Military,Initial Approach,Government,Approach,1299098,Conflict Airborne Conflict; Deviation - Speed ...,Person Air Traffic Control,Human Factors,I was sequencing multiple arrivals to ZZZ; 2 r...,A Controller was vectoring a VFR flight of fou...
2,1299106,201510,ZMA.ARTCC,FL,Government,Medium Transport; Low Wing; 2 Turbojet Eng,Cruise,Government,Handoff / Assist; Oceanic,1299106,Airspace Violation All Types; Deviation / Disc...,Person Air Traffic Control,Procedure,An aircraft entered New York's non-radar airsp...,ARTCC Controller reported relieving a position...
3,1299250,201510,ZZZ.ARTCC,US,Air Carrier,A319,Cruise,Air Carrier,Pilot Not Flying; First Officer,1299250,Deviation - Altitude Excursion From Assigned A...,Person Flight Crew,Weather,We were cruising at FL360; in cirrus type clou...,A319 Flight Crew experienced turbulence in cir...
4,1299290,201510,ZZZ.Airport,US,Air Carrier,MD-11,Initial Approach,Air Carrier,Captain,1299290,Aircraft Equipment Problem Less Severe; Inflig...,Person Flight Crew,Environment - Non Weather Related,Bird strike at approximately 300-400 AGL durin...,On short final; an MD-11 was impacted by a bir...


In [78]:
primary_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212322 entries, 0 to 4732
Data columns (total 15 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   ACN                                  212322 non-null  int64 
 1   Date                                 212310 non-null  object
 2   Locale Reference                     206590 non-null  object
 3   State Reference                      208458 non-null  object
 4   Aircraft Operator                    202231 non-null  object
 5   Make Model Name                      208108 non-null  object
 6   Flight Phase                         204024 non-null  object
 7   Reporter Organization                210199 non-null  object
 8   Function                             210346 non-null  object
 9   ASRS Report Number.Accession Number  211281 non-null  object
 10  Anomaly                              211531 non-null  object
 11  Detector                         

In [79]:
primary_df["Date"] = pd.to_datetime(primary_df["Date"], format="%Y%m", errors="coerce")
primary_df = primary_df.astype({"ACN": "uint32"})

/var/folders/w4/pt773y8s3b736sz4g0jqyc180000gn/T/ipykernel_27360/1890960120.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_df["Date"] = pd.to_datetime(primary_df["Date"], format="%Y%m", errors="coerce")


In [80]:
primary_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212322 entries, 0 to 4732
Data columns (total 15 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   ACN                                  212322 non-null  uint32        
 1   Date                                 208368 non-null  datetime64[ns]
 2   Locale Reference                     206590 non-null  object        
 3   State Reference                      208458 non-null  object        
 4   Aircraft Operator                    202231 non-null  object        
 5   Make Model Name                      208108 non-null  object        
 6   Flight Phase                         204024 non-null  object        
 7   Reporter Organization                210199 non-null  object        
 8   Function                             210346 non-null  object        
 9   ASRS Report Number.Accession Number  211281 non-null  object        
 10  Ano

In [82]:
primary_df = primary_df.drop(labels=["ASRS Report Number.Accession Number"], axis=1)

In [86]:
primary_df.columns = [
    col.lower().strip().replace(" ", "_") for col in primary_df.columns
]

In [87]:
primary_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 212322 entries, 0 to 4732
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   acn                    212322 non-null  uint32        
 1   date                   208368 non-null  datetime64[ns]
 2   locale_reference       206590 non-null  object        
 3   state_reference        208458 non-null  object        
 4   aircraft_operator      202231 non-null  object        
 5   make_model_name        208108 non-null  object        
 6   flight_phase           204024 non-null  object        
 7   reporter_organization  210199 non-null  object        
 8   function               210346 non-null  object        
 9   anomaly                211531 non-null  object        
 10  detector               207512 non-null  object        
 11  primary_problem        203493 non-null  object        
 12  narrative              212287 non-null  object     

In [ ]:
primary_df.to_parquet("../data/01_primary/asrs_data_primary.parquet", index=False)

In [ ]:
primary_df.to_parquet(
    "../data/01_primary/asrs_data_primary.parquet.gzip", index=False, compression="gzip"
)